In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
from bs4 import BeautifulSoup
import re
import Levenshtein as lv
import time as timer
import os
import pandas as pd

In [3]:
# 파일 경로
driver_path = 'chromedriver-win64\\chromedriver.exe'

# 파일 존재 여부 확인
if os.path.exists(driver_path):
    print("파일 존재.")
else:
    print("파일 없음.")

파일 존재.


In [4]:
def calLocation(search_name, target) :
    for index, t in enumerate(target) :
        if(search_name in t):
            return index
        
    return -1

In [5]:
def jaccard_similarity(str1, str2):
    # 두 문자열을 각각의 문자열로 분해하여 집합으로 변환
    set1 = set(str1)
    set2 = set(str2)
    
    # 교집합과 합집합의 크기 계산
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    
    # 자카드 유사도 계산
    similarity = intersection / union
    
    return similarity

In [6]:
# CSV 파일 읽기
df = pd.read_csv('hihi.csv')

print(df['업소명'].size)
print(df['세부주소'].size)

csv_address = df['세부주소'][0]
print(csv_address)

999
999
종로구


In [21]:
browser = webdriver.Chrome(driver_path)

searchTitle = "https://map.naver.com/p?c=15.00,0,0,0,dh"

MAX_WAIT_TIME = 3

browser.implicitly_wait(MAX_WAIT_TIME)

with open('result.csv', 'w', newline='', encoding='utf-8') as file:
    # CSV writer 객체 생성
    writer = csv.DictWriter(file, fieldnames=['업소명', '세부주소', '지도 업소명', '카테고리', '메뉴', '태그'])
    # CSV 파일에 헤더 쓰기
    writer.writeheader()

    browser.get(searchTitle)

    d = df['업소명'].size

    for i in range(0, d) :
        #csv 파일에서 업소 명, 도로명 주소 불러와서 사용
        csv_title = df['업소명'][i]
        csv_address = df['세부주소'][i]

        print(f"({i + 1}/{d}) - {csv_title} - " , end="")

        browser.switch_to.default_content()

        # 검색창에 주소 검색
        try:
            try:
                search_box = WebDriverWait(browser, MAX_WAIT_TIME).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "input_search"))
                )
                
                search_box.send_keys(Keys.CONTROL + "a")
                search_box.send_keys(Keys.DELETE)
                search_box.send_keys(csv_title + Keys.ENTER)

            #예외 처리
            except TimeoutException:
                print("검색창 탐색 실패 - 저장 실패")
                continue

            flag = True

            try:
                iframe = browser.find_element_by_id('entryIframe')
                browser.switch_to.frame(iframe)
                flag = False
            except NoSuchElementException:
                browser.switch_to.default_content()
                try :
                    secondIframe = browser.find_element_by_id('searchIframe')
                    browser.switch_to.frame(secondIframe)
                except NoSuchElementException :
                    print("searchIframe 탐색 실패 - 저장 실패")
                    continue

            if(flag) :
                address_box = []

                address_container = browser.find_elements_by_class_name('Pb4bU')
                if(len(address_container) > 0):
                    for address in address_container:
                        address_box.append(address.text)
                else :
                    print("검색 결과 없음 - 저장 실패")
                    continue

                target_index = calLocation(csv_address, address_box)

                if(target_index == -1) :
                    print("일치 요소 없음 - 저장 실패")
                    continue

                title_anchor = browser.find_elements_by_class_name("ApCpt")
                try : 
                    if(len(title_anchor) > 0):
                        title_anchor[target_index].click()
                    else :
                        print("클릭 실패 - 저장 실패")
                        continue
                except ElementClickInterceptedException:
                    print("클릭 요소 가림 - 저장 실패")
                    continue

                browser.switch_to.default_content()

                try:
                    iframe2 = browser.find_element_by_id("entryIframe")
                    browser.switch_to.frame(iframe2)
                except NoSuchElementException:
                    print("entryIframe 탐색 실패 - 저장 실패")
                    continue
            
            title_text = ""

            title_temp = browser.find_elements_by_class_name("Fc1rA")
            if(len(title_temp) > 0) :
                title_text = title_temp[0].text

            category_text = ""

            category_temp = browser.find_elements_by_class_name("DJJvD")
            if(len(category_temp) > 0) :
                category_text = category_temp[0].text

            # # 별점
            # score_elements = browser.find_elements_by_class_name("PXMot.LXIwF")
            # score_number = re.sub('[^0-9.]', '', score_elements[0].text) if score_elements else ''

            # # 방문자 리뷰
            # review_elements = browser.find_elements_by_xpath('//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[2]/a')
            # review_number = re.sub('[^0-9.]', '', review_elements[0].text) if review_elements else ''

            # # 블로그 리뷰
            # blog_elements = browser.find_elements_by_xpath('//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[3]/a')
            # blog_number = re.sub('[^0-9.]', '', blog_elements[0].text) if blog_elements else ''

            # schedule = []

            # try :
            #     time_button = browser.find_element_by_xpath('//*[@id="app-root"]/div/div/div/div[5]/div/div[2]/div[1]/div/div[3]/div/a')
            #     time_button.click()

            #     day_box = browser.find_elements_by_class_name('i8cJw')
            #     time_box = browser.find_elements_by_class_name('H3ua4')

            #     for (day, t) in zip(day_box, time_box):
            #         schedule.append((day.text, t.text))

            # except NoSuchElementException:
            #     continue

            menu_list = ""
            list_box = []

            try :
                list_button = browser.find_elements_by_class_name('veBoZ')
                
                if(len(list_button) > 0):
                    for m in list_button:
                        list_box.append(m.text)
                else :
                    print("정보 없음 - 저장 실패")
                    continue

                target_index = calLocation("메뉴", list_box)

                if(target_index == -1) :
                    print("일치 요소 없음 - 저장 실패")
                    continue
                else :
                    list_button[target_index].click()

                menu_box = []

                menu_container = browser.find_elements_by_class_name('lPzHi')
                if(len(menu_container) > 0):
                    for m in menu_container:
                        menu_box.append(m.text)

                for index, menu in enumerate(menu_box):
                    if(index == 0) :
                        menu_list += "[" + menu + "]"
                    else :
                        menu_list += ", [" + menu + "]"

            except NoSuchElementException:
                print("메뉴 없음 ㅆㅃ")
                continue

            target_index = calLocation("리뷰", list_box)

            if(target_index == -1) :
                print("일치 요소 없음 - 저장 실패")
                continue
            else : 
                list_button[target_index].click()

            while(True) :
                detail_button = browser.find_elements_by_class_name("dP0sq")

                if(len(detail_button) > 0):
                    detail_button[0].click()
                else :
                    break

            tag_list = ""
            tag_box = []
            tagCount_box = []

            tag_container = browser.find_elements_by_class_name("t3JSf")

            if(len(tag_container) > 0):
                for tag in tag_container:
                    tag_box.append(tag.text)
            else :
                print("태그 없음 - 저장 실패")
                continue

            tagCount_container = browser.find_elements_by_class_name("CUoLy")

            if(len(tagCount_container) > 0):
                for tagCount in tagCount_container:
                    tagCount_box.append(tagCount.text)
            else :
                print("태그 수 없음 - 저장 실패")
                continue

            for index, (tag, tagCount) in enumerate(zip(tag_box, tagCount_box)):
                tag_number = re.sub("[^0-9]", "", tagCount)
                if(index == 0) :
                    tag_list += "[" + tag + "," + tag_number + "]"
                else :
                    tag_list += ", [" + tag + "," + tag_number + "]"

            data = {
                '업소명' : csv_title,
                '세부주소' : csv_address,
                '지도 업소명' : title_text,
                '카테고리' : category_text,
                # '별점' : score_number,
                # '방문자리뷰' : review_number,
                # '블로그리뷰' : review_number,
                '메뉴' : menu_list,
                '태그' : tag_list
            }

            # if(schedule) :
            #     for day, t in schedule:
            #         data[day] = t

            writer.writerow(data)

            print("처리 완료")
        except StaleElementReferenceException:
            print("에러 발생 - 저장 실패")
            continue
    # 작업 완료 후 드라이버 종료
    browser.quit()

(1/999) - 남대문막내횟집 - 처리 완료
(2/999) - 가봉루 - 처리 완료
(3/999) - 백제고기나라 - 검색 결과 없음 - 저장 실패
(4/999) - 인사동.한정식.둘레골 - 처리 완료
(5/999) - 조금 - 처리 완료
(6/999) - 솔밭숯불갈비 - 일치 요소 없음 - 저장 실패
(7/999) - 양반댁 - 처리 완료
(8/999) - 조마루(신설동점) - 클릭 실패 - 저장 실패
(9/999) - 한라의 집 - 검색 결과 없음 - 저장 실패
(10/999) - 모토이시 종각점 - 클릭 실패 - 저장 실패
(11/999) - 하나로회관 - 처리 완료
(12/999) - 김가네대청마루 - 처리 완료
(13/999) - 동북풍미분식 - 검색 결과 없음 - 저장 실패
(14/999) - 산동 - searchIframe 탐색 실패 - 저장 실패
(15/999) - 박가네빈대떡 2호점 - 정보 없음 - 저장 실패
(16/999) - 할머니두부집 - 처리 완료
(17/999) - 명동 면옥 - 일치 요소 없음 - 저장 실패
(18/999) - 궁안에 개성만두 - 검색 결과 없음 - 저장 실패
(19/999) - 전주골갈비 - 처리 완료
(20/999) - 도도야 - 처리 완료
(21/999) - 궁나라냉면.묵밥 - 처리 완료
(22/999) - 통나무집 - 처리 완료
(23/999) - 종로진낙지 - 처리 완료
(24/999) - 뉘조 - 처리 완료
(25/999) - 백부장집 2호점 - 처리 완료
(26/999) - 안래홍 - 처리 완료
(27/999) - 종각한우 - 처리 완료
(28/999) - 인사동두부마을 - 처리 완료
(29/999) - 옥정 - searchIframe 탐색 실패 - 저장 실패
(30/999) - 북촌진곰탕 - 처리 완료
(31/999) - 마산해물아구찜 - 처리 완료
(32/999) - 사동면옥 - 처리 완료
(33/999) - 종로술집 - 클릭 실패 - 저장 실패
(34/999) - 황금코다리 - 일치 요소 없음 -

ElementClickInterceptedException: Message: element click intercepted: Element <iframe id="entryIframe" title="Naver Place Entry" src="https://pcmap.place.naver.com/place/37233200?entry=bmp&amp;from=map&amp;fromPanelNum=2&amp;timestamp=202403201657" cd_frame_id_="32bb8a45e18cef26b5c01e5e0e6d4e61"></iframe> is not clickable at point (178, 513). Other element would receive the click: <iframe id="searchIframe" title="Naver Place Search" src="https://pcmap.place.naver.com/place/list?query=%EC%A7%84%EC%B0%B8%EC%B9%98&amp;x=126.94902337490294&amp;y=37.541135699999415&amp;clientX=127.060889&amp;clientY=36.798542&amp;bounds=126.93844474254922%3B37.53449138499076%3B126.95981658398165%3B37.54760929658468&amp;ts=1710921427422&amp;mapUrl=https%3A%2F%2Fmap.naver.com%2Fp%2Fsearch%2F%EC%A7%84%EC%B0%B8%EC%B9%98" cd_frame_id_="32e1e6986ca57cbacd6920cf90d88b2d"></iframe>
  (Session info: chrome=122.0.6261.129)


In [28]:
df = pd.read_csv('result(need_retouch).csv')

df['태그'] = df['태그'].str.replace(r"[\[\]',.%\" ]", "", regex=True)  # 대괄호와 작은따옴표 삭제
df['태그'] = df['태그'].str.replace(r"\d", "", regex=True)  

df.to_csv('result(need_retouch).csv', index=False)

df = pd.read_csv('result(need_retouch).csv')

print(df['태그'])

0         #가성비#양많음#친절#독특함
1         #양많음#친절#가성비#독특함
2          #단체#친절#분위기#분위기
3          #독특함#양많음#혼밥#친절
4              #친절#단체#분위기
              ...        
339       #혼밥#가성비#친절#넓은매장
340       #친절#양많음#독특함#가성비
341    #친절#청결#넓은매장#양많음#혼밥
342            #친절#독특함#단체
343      #분위기#독특함#친절#넓은매장
Name: 태그, Length: 344, dtype: object


In [29]:
df = pd.read_csv('result(need_retouch).csv')

df['태그'] = df['태그'].str.replace(r"#", ", #", regex=True)

df.to_csv('result(need_retouch).csv', index=False)

df = pd.read_csv('result(need_retouch).csv')

print(df['태그'])

0           , #가성비, #양많음, #친절, #독특함
1           , #양많음, #친절, #가성비, #독특함
2            , #단체, #친절, #분위기, #분위기
3            , #독특함, #양많음, #혼밥, #친절
4                  , #친절, #단체, #분위기
                   ...             
339         , #혼밥, #가성비, #친절, #넓은매장
340         , #친절, #양많음, #독특함, #가성비
341    , #친절, #청결, #넓은매장, #양많음, #혼밥
342                , #친절, #독특함, #단체
343        , #분위기, #독특함, #친절, #넓은매장
Name: 태그, Length: 344, dtype: object


In [30]:
df = pd.read_csv('result(need_retouch).csv')

def clean_first_comma_and_space(tag_str):
    # 첫 번째 쉼표와 첫 번째 공백을 제거합니다.
    if tag_str.startswith(','):
        tag_str = tag_str[1:]  # 첫 번째 쉼표 제거
    tag_str = tag_str.lstrip(' ')  # 첫 번째 공백 제거
    return tag_str

# 각 행에 대해 청소 함수를 적용합니다.
df['태그'] = df['태그'].apply(clean_first_comma_and_space)

df.to_csv('result(need_retouch).csv', index=False)

df = pd.read_csv('result(need_retouch).csv')

print(df['태그'])

0           #가성비, #양많음, #친절, #독특함
1           #양많음, #친절, #가성비, #독특함
2            #단체, #친절, #분위기, #분위기
3            #독특함, #양많음, #혼밥, #친절
4                  #친절, #단체, #분위기
                  ...            
339         #혼밥, #가성비, #친절, #넓은매장
340         #친절, #양많음, #독특함, #가성비
341    #친절, #청결, #넓은매장, #양많음, #혼밥
342                #친절, #독특함, #단체
343        #분위기, #독특함, #친절, #넓은매장
Name: 태그, Length: 344, dtype: object


In [5]:
df = pd.read_csv('result(ing).csv')

df['메뉴'] = df['메뉴'].str.replace(r"[\[\]]", "", regex=True)

df.to_csv('result(ing).csv', index=False)

df = pd.read_csv('result(ing).csv')

print(df['메뉴'])

0      회정식+매운탕(점심특선), 모둠(대), 회덮밥+매운탕(점심특선), 알밥+매운탕(점심...
1                                                    라조위
2       점심특선A(평일), 점심특선B(주말), 보리굴비정식, 궁특선, 별정식, 달정식, 해정식
3                         조금솥밥, 은행구이, 양송이구이, 전복솥밥, 양송이솥밥
4                    간장게장 솥밥정식, 보리굴비 솥밥정식, 국내산 돼지갈비 솥밥정식
                             ...                        
339                                            순대국, 뼈해장국
340                       급랭삼겹살, 제주흑돼지 꼬들살, 생삼겹살, 급랭 꽃목살
341    설렁탕, 왕갈비탕, 특 설렁탕, 소머리국밥, 도가니탕, 수육(小), 수육(大), 모...
342    예약 코스메뉴, 접시만두, 한우제물국시, 만두전골, 낙지파전, 모듬전, 만둣국, 떡...
343    런치세트, 반반피자, 갈릭바베큐치킨피자, 화이트트럼플머쉬룸피자, 베이컨 하와이안, ...
Name: 메뉴, Length: 344, dtype: object
